# <center>JSONiq Update Facility sandbox</center>


The following cells are meant for Google Colab, which does not have the right Java version.

In [ ]:
# We make sure Java 17 is installed.
!apt update
!apt install openjdk-17-jdk
!java -version

In [2]:
!pip install jsoniq

Found existing installation: jsoniq 0.2.0a8
Uninstalling jsoniq-0.2.0a8:
  Successfully uninstalled jsoniq-0.2.0a8
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.8/27.8 MB 6.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


We now initialize a RumbleDB session and test that it works.

In [3]:
from jsoniq import RumbleSession

rumble = RumbleSession.builder.withDelta().getOrCreate();

print(rumble.jsoniq('{ "foo": [ 6*7 ] }').json());

:: loading settings :: url = jar:file:/Users/ghislain/.pyenv/versions/3.11.9/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/ghislain/.ivy2.5.2/cache
The jars for the packages stored in: /Users/ghislain/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-19995126-c77c-4a37-b90d-2b3665df98b0;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in local-m2-cache
	found io.delta#delta-storage;4.0.0 in local-m2-cache
	found org.antlr#antlr4-runtime;4.13.1 in local-m2-cache
:: resolution report :: resolve 71ms :: artifacts dl 2ms
	:: modules in use:
	io.delta#delta-spark_2.13;4.0.0 from local-m2-cache in [default]
	io.delta#delta-storage;4.0.0 from local-m2-cache in [default]
	org.antlr#antlr4-runtime;4.13.1 from local-m2-cache in [default]
	---------------------------------------------------------------------
	|         

({'foo': [42]},)


For convenience, we also activate the JSONiq magic.

In [4]:
%load_ext jsoniqmagic
# We will display the first 5 items. This parameter can be changed.
rumble.getRumbleConf().setResultSizeCap(20)

JavaObject id=o46

We create a new collection

In [7]:
%%jsoniq -u
create collection table("foo") with for $i in 1 to 3 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }


Updates applied successfully.


In [8]:
%%jsoniq -df
table("foo")

+---+--------------------+-------+----------+-----------------+--------+---------------+
|foo|                 bar|__rowID|__rowOrder|__mutabilityLevel|__pathIn|__tableLocation|
+---+--------------------+-------+----------+-----------------+--------+---------------+
|  1|[Alice, Bob, Char...|      0|       0.0|                0|        |            foo|
|  2|[Alice, Bob, Char...|      1|       1.0|                0|        |            foo|
|  3|[Alice, Bob, Char...|      2|       2.0|                0|        |            foo|
+---+--------------------+-------+----------+-----------------+--------+---------------+



We insert new data -- remember the snapshot semantics, all updates are computed against the original table, and then applied jointly.

In [9]:
%%jsoniq -u
insert for $i in 4 to 6 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  first into collection table("foo"),
insert for $i in 7 to 10 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  last into collection table("foo"),
insert for $i in 11 to 13 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  before table("foo")[position() eq 2] into collection,
insert for $i in 14 to 16 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  after table("foo")[position() eq 2] into collection


Updates applied successfully.


In [10]:
%%jsoniq -df
table("foo")

+---+--------------------+-------+----------+-----------------+--------+---------------+
|foo|                 bar|__rowID|__rowOrder|__mutabilityLevel|__pathIn|__tableLocation|
+---+--------------------+-------+----------+-----------------+--------+---------------+
|  4|[Alice, Bob, Char...|      9|  -75000.0|                0|        |            foo|
|  5|[Alice, Bob, Char...|     10|  -50000.0|                0|        |            foo|
|  6|[Alice, Bob, Char...|     11|  -25000.0|                0|        |            foo|
|  1|[Alice, Bob, Char...|      0|       0.0|                0|        |            foo|
| 11|[Alice, Bob, Char...|      3|      0.25|                0|        |            foo|
| 12|[Alice, Bob, Char...|      4|       0.5|                0|        |            foo|
| 13|[Alice, Bob, Char...|      5|      0.75|                0|        |            foo|
|  2|[Alice, Bob, Char...|      1|       1.0|                0|        |            foo|
| 14|[Alice, Bob, Cha

We delete a record

In [11]:
%%jsoniq -u
delete table("foo")[$$.foo eq 15] from collection

Updates applied successfully.


In [12]:
%%jsoniq -df
table("foo")

+---+--------------------+-------+----------+-----------------+--------+---------------+
|foo|                 bar|__rowID|__rowOrder|__mutabilityLevel|__pathIn|__tableLocation|
+---+--------------------+-------+----------+-----------------+--------+---------------+
|  4|[Alice, Bob, Char...|      9|  -75000.0|                0|        |            foo|
|  5|[Alice, Bob, Char...|     10|  -50000.0|                0|        |            foo|
|  6|[Alice, Bob, Char...|     11|  -25000.0|                0|        |            foo|
|  1|[Alice, Bob, Char...|      0|       0.0|                0|        |            foo|
| 11|[Alice, Bob, Char...|      3|      0.25|                0|        |            foo|
| 12|[Alice, Bob, Char...|      4|       0.5|                0|        |            foo|
| 13|[Alice, Bob, Char...|      5|      0.75|                0|        |            foo|
|  2|[Alice, Bob, Char...|      1|       1.0|                0|        |            foo|
| 14|[Alice, Bob, Cha

We edit a record

In [13]:
%%jsoniq -u
edit table("foo")[$$.foo eq 14] into { "foo" : 17, "bar" : [ "Alice", "Bob", "Daniel" ] } in collection

Updates applied successfully.


In [14]:
%%jsoniq -df
table("foo")

+---+--------------------+-------+----------+-----------------+--------+---------------+
|foo|                 bar|__rowID|__rowOrder|__mutabilityLevel|__pathIn|__tableLocation|
+---+--------------------+-------+----------+-----------------+--------+---------------+
|  4|[Alice, Bob, Char...|      9|  -75000.0|                0|        |            foo|
|  5|[Alice, Bob, Char...|     10|  -50000.0|                0|        |            foo|
|  6|[Alice, Bob, Char...|     11|  -25000.0|                0|        |            foo|
|  1|[Alice, Bob, Char...|      0|       0.0|                0|        |            foo|
| 11|[Alice, Bob, Char...|      3|      0.25|                0|        |            foo|
| 12|[Alice, Bob, Char...|      4|       0.5|                0|        |            foo|
| 13|[Alice, Bob, Char...|      5|      0.75|                0|        |            foo|
|  2|[Alice, Bob, Char...|      1|       1.0|                0|        |            foo|
| 17|[Alice, Bob, Dan

In [15]:
%%jsoniq
rename json table("foo")[1].foo as "bar2";

In [19]:
%%jsoniq -df
table("foo")

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `foo` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01;
'UnresolvedRelation [foo], [], false


In [18]:
%%jsoniq -u
delete collection table("foo")

Updates applied successfully.


In [33]:
%%jsoniq -u
create collection delta-file("foobar") with for $i in 1 to 3 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }


Updates applied successfully.


In [34]:
%%jsoniq -df
delta-file("foobar")

+---+--------------------+-------+----------+-----------------+--------+--------------------+
|foo|                 bar|__rowID|__rowOrder|__mutabilityLevel|__pathIn|     __tableLocation|
+---+--------------------+-------+----------+-----------------+--------+--------------------+
|  1|[Alice, Bob, Char...|      0|       0.0|                0|        |delta.`file:/User...|
|  2|[Alice, Bob, Char...|      1|       1.0|                0|        |delta.`file:/User...|
|  3|[Alice, Bob, Char...|      2|       2.0|                0|        |delta.`file:/User...|
+---+--------------------+-------+----------+-----------------+--------+--------------------+



In [38]:
%%jsoniq -u
insert for $i in 4 to 6 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  first into collection delta-file("foobar"),
insert for $i in 7 to 10 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  last into collection delta-file("foobar"),
insert for $i in 11 to 13 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  before delta-file("foobar")[position() eq 2] into collection,
insert for $i in 14 to 16 return { "foobar" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  after delta-file("foobar")[position() eq 2] into collection


Updates applied successfully.


In [39]:
%%jsoniq -df
delta-file("foobar")

+---+--------------------+-------+----------+-----------------+--------+--------------------+
|foo|                 bar|__rowID|__rowOrder|__mutabilityLevel|__pathIn|     __tableLocation|
+---+--------------------+-------+----------+-----------------+--------+--------------------+
|  4|[Alice, Bob, Char...|      9|  -75000.0|                0|        |delta.`file:/User...|
|  5|[Alice, Bob, Char...|     10|  -50000.0|                0|        |delta.`file:/User...|
|  6|[Alice, Bob, Char...|     11|  -25000.0|                0|        |delta.`file:/User...|
|  1|[Alice, Bob, Char...|      0|       0.0|                0|        |delta.`file:/User...|
| 11|[Alice, Bob, Char...|      3|      0.25|                0|        |delta.`file:/User...|
| 12|[Alice, Bob, Char...|      4|       0.5|                0|        |delta.`file:/User...|
| 13|[Alice, Bob, Char...|      5|      0.75|                0|        |delta.`file:/User...|
|  2|[Alice, Bob, Char...|      1|       1.0|               

In [40]:
%%jsoniq -u
delete delta-file("foobar")[$$.foo eq 15] from collection

Updates applied successfully.


In [41]:
%%jsoniq -df
delta-file("foobar")

+---+--------------------+-------+----------+-----------------+--------+--------------------+
|foo|                 bar|__rowID|__rowOrder|__mutabilityLevel|__pathIn|     __tableLocation|
+---+--------------------+-------+----------+-----------------+--------+--------------------+
|  4|[Alice, Bob, Char...|      9|  -75000.0|                0|        |delta.`file:/User...|
|  5|[Alice, Bob, Char...|     10|  -50000.0|                0|        |delta.`file:/User...|
|  6|[Alice, Bob, Char...|     11|  -25000.0|                0|        |delta.`file:/User...|
|  1|[Alice, Bob, Char...|      0|       0.0|                0|        |delta.`file:/User...|
| 11|[Alice, Bob, Char...|      3|      0.25|                0|        |delta.`file:/User...|
| 12|[Alice, Bob, Char...|      4|       0.5|                0|        |delta.`file:/User...|
| 13|[Alice, Bob, Char...|      5|      0.75|                0|        |delta.`file:/User...|
|  2|[Alice, Bob, Char...|      1|       1.0|               

In [43]:
%%jsoniq -u
edit delta-file("foobar")[$$.foo eq 14] into { "foo" : 17, "bar" : [ "Alice", "Bob", "Daniel" ] } in collection

Updates applied successfully.


In [44]:
%%jsoniq -df
delta-file("foobar")

+---+--------------------+-------+----------+-----------------+--------+--------------------+
|foo|                 bar|__rowID|__rowOrder|__mutabilityLevel|__pathIn|     __tableLocation|
+---+--------------------+-------+----------+-----------------+--------+--------------------+
|  4|[Alice, Bob, Char...|      9|  -75000.0|                0|        |delta.`file:/User...|
|  5|[Alice, Bob, Char...|     10|  -50000.0|                0|        |delta.`file:/User...|
|  6|[Alice, Bob, Char...|     11|  -25000.0|                0|        |delta.`file:/User...|
|  1|[Alice, Bob, Char...|      0|       0.0|                0|        |delta.`file:/User...|
| 11|[Alice, Bob, Char...|      3|      0.25|                0|        |delta.`file:/User...|
| 12|[Alice, Bob, Char...|      4|       0.5|                0|        |delta.`file:/User...|
| 13|[Alice, Bob, Char...|      5|      0.75|                0|        |delta.`file:/User...|
|  2|[Alice, Bob, Char...|      1|       1.0|               

In [32]:
%%jsoniq -u
delete collection delta-file("foobar")

Updates applied successfully.
